In [1]:
!pip install python-telegram-bot gspread oauth2client


In [2]:
!pip install nest_asyncio


In [3]:
!pip install pytelegrambotapi

In [ ]:
import telebot
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Настройка доступа к Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("#############################.json", scope)
client = gspread.authorize(creds)

# Откроем таблицу по названию
sheet = client.open("Leads Submissions").sheet1

# Инициализация бота
bot = telebot.TeleBot('#####################################')

# Доступные языки
LANGUAGES = {
    'ru': 'Русский',
    'en': 'English',
    'he': 'עברית'
}

# Состояния пользователя
user_state = {}

# Сообщения на разных языках
MESSAGES = {
    'ru': {
        'welcome': "Привет! Выберите язык / Choose a language / בחר שפה:",
        'name_prompt': "Пожалуйста, укажите ваше имя.",
        'service_prompt': "Какая услуга вас интересует?",
        'service_options': ["ВебСайт", "Продвижение и Реклама", "Консультация"],
        'sector_prompt': "Пожалуйста, укажите сферу вашей деятельности.",
        'budget_prompt': "Каков ваш бюджет?",
        'budget_options': ["500-1,000 $", "1,000-3,000 $", "3,000-5,000 $", "Другое"],
        'phone_prompt': "Пожалуйста, укажите ваш номер телефона.",
        'thanks': "Спасибо! Ваша заявка принята.",
        'additional_info': "Спасибо за дополнительную информацию, мы учтем её и свяжемся с вами.",
        'phone_invalid': 'Неверный номер телефона. Введите заново (только цифры).'
    },
    'en': {
        'welcome': "Hello! Please choose a language / Выберите язык / בחר שפה:",
        'name_prompt': "Please provide your name.",
        'service_prompt': "What service are you interested in?",
        'service_options': ["Website", "Promotion and Advertising", "Consultation"],
        'sector_prompt': "Please specify your business sector.",
        'budget_prompt': "What is your budget?",
        'budget_options': ["500-1,000 $", "1,000-3,000 $", "3,000-5,000 $", "Other"],
        'phone_prompt': "Please provide your phone number.",
        'thanks': "Thank you! Your request has been submitted.",
        'additional_info': "Thank you for the additional information, we'll consider it and get in touch with you.",
        'phone_invalid': "Invalid phone number. Please enter it again (digits only)."
    },
    'he': {
        'welcome': "שלום! אנא בחר שפה / Choose a language / Выберите язык:",
        'name_prompt': "מה שמך?",
        'service_prompt': "באיזה שירות מעניין\ת?",
        'service_options': ["אתר אינטרנט", "קידום ופרסום", "ייעוץ"],
        'sector_prompt': "אנא ציין את תחום העיסוק שלך.",
        'budget_prompt': "מה התקציב שלך?",
        'budget_options': ["500-1,000 ($)", "1,000-3,000 ($)", "3,000-5,000 ($)", "אחר"],
        'phone_prompt': "אנא ספק את מספר הטלפון שלך.",
        'thanks': "תודה! בקשתך התקבלה.",
        'additional_info': "תודה על המידע הנוסף, ניקח זאת בחשבון וניצור איתך קשר.",
        'phone_invalid': "מספר טלפון לא חוקי. אנא הכנס שוב (מספרים בלבד)."
    }
}

# Начало диалога
@bot.message_handler(commands=['start'])
def start(message):
    user_id = message.chat.id
    user_state[user_id] = {'step': 'choose_language'}
    
    # Отправляем приветствие
    keyboard = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for lang in LANGUAGES.values():
        keyboard.add(lang)
    bot.send_message(user_id, "Hello! Please choose a language / Выберите язык / בחר שפה:", reply_markup=keyboard)
    
# Функция для проверки корректности номера телефона
def is_valid_phone_number(phone):
    return bool(re.match(r'^\+?\d+$', phone))
    
# Обработка сообщений пользователя
@bot.message_handler(content_types=['text'])
def handle_text(message):
    user_id = message.chat.id
    state = user_state.get(user_id, {})
    step = state.get('step')
    
    # Получение username и формирование ссылки на профиль
    username = message.from_user.username
    if username:
        profile_link = f"https://t.me/{username}"
    else:
        profile_link = f"https://t.me/user?id={user_id}"
        
    # Шаг выбора языка
    if step == 'choose_language':
        for code, lang in LANGUAGES.items():
            if message.text == lang:
                user_state[user_id]['language'] = code
                user_state[user_id]['step'] = 'get_name'
                bot.send_message(user_id, MESSAGES[code]['name_prompt'])
                return
        
        bot.send_message(user_id, "Выберите язык из предложенных вариантов.")
    
    # Шаг ввода имени
    elif step == 'get_name':
        user_state[user_id]['name'] = message.text
        user_state[user_id]['step'] = 'get_service'
        lang_code = user_state[user_id]['language']
        
        # Сохраняем имя в таблицу
        sheet.append_row([message.text, message.from_user.username or ""])
        
        # Переход к выбору услуги
        keyboard = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
        for service in MESSAGES[lang_code]['service_options']:
            keyboard.add(service)
        bot.send_message(user_id, MESSAGES[lang_code]['service_prompt'], reply_markup=keyboard)
    
    # Шаг выбора услуги
    elif step == 'get_service':
        user_state[user_id]['service'] = message.text
        user_state[user_id]['step'] = 'get_sector'
        lang_code = user_state[user_id]['language']
        bot.send_message(user_id, MESSAGES[lang_code]['sector_prompt'])
    
    # Шаг ввода сферы деятельности
    elif step == 'get_sector':
        user_state[user_id]['sector'] = message.text
        user_state[user_id]['step'] = 'get_budget'
        lang_code = user_state[user_id]['language']
        
        # Переход к выбору бюджета
        keyboard = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
        for budget in MESSAGES[lang_code]['budget_options']:
            keyboard.add(budget)
        bot.send_message(user_id, MESSAGES[lang_code]['budget_prompt'], reply_markup=keyboard)
    
    # Шаг выбора бюджета
    elif step == 'get_budget':
        user_state[user_id]['budget'] = message.text
        user_state[user_id]['step'] = 'get_phone'
        lang_code = user_state[user_id]['language']
        bot.send_message(user_id, MESSAGES[lang_code]['phone_prompt'])
    
    # Шаг ввода номера телефона с проверкой
    elif step == 'get_phone':
        if is_valid_phone_number(message.text):
            user_state[user_id]['phone'] = message.text
            lang_code = user_state[user_id]['language']
            
            # Сохраняем все данные в Google Sheets, включая ссылку на профиль
            sheet.append_row([
                user_state[user_id].get('name', ''),
                user_state[user_id].get('service', ''),
                user_state[user_id].get('sector', ''),
                user_state[user_id].get('budget', ''),
                user_state[user_id].get('phone', ''),
                profile_link,  # Ссылка на профиль пользователя
                user_state[user_id].get('additional_info', '')
            ])
            
            # Завершение
            bot.send_message(user_id, MESSAGES[lang_code]['thanks'])
            user_state[user_id]['step'] = 'completed'
        else:
            # Сообщение о некорректном номере телефона
            lang_code = user_state[user_id]['language']
            bot.send_message(user_id, MESSAGES[lang_code]['phone_invalid'])
            # Оставляем шаг на 'get_phone' для повторного ввода номера
            user_state[user_id]['step'] = 'get_phone'
    
# Сбор дополнительной информации после завершения
    elif step == 'completed':
        lang_code = user_state[user_id]['language']
        additional_info = user_state[user_id].get('additional_info', '')
        additional_info += f"{message.text}\n"  # Добавляем информацию с новой строки
        user_state[user_id]['additional_info'] = additional_info
    
    # Обновляем данные в Google Sheets с дополнительной информацией, добавляя в последнюю строку
        try:
            last_row = len(sheet.get_all_values())  # Получаем количество строк
            sheet.update_cell(last_row, 7, additional_info)  # Обновляем ячейку в 7 колонке
        
        # Отправляем сообщение пользователю
            bot.send_message(user_id, MESSAGES[lang_code]['additional_info'])
        except Exception as e:
            print(f"Ошибка при обновлении таблицы: {e}")
        
# Запуск бота
bot.polling(none_stop=True)

